# Import

These requirements are necessary if you launch this notebook from SageMaker instances

In [1]:
"""!pip install mlflow
!pip install pytorch-lightning
!pip install transformers
!pip install tqdm
!pip install sagemaker

!pip install s3fs
!pip install smdebug"""

'!pip install mlflow\n!pip install pytorch-lightning\n!pip install transformers\n!pip install tqdm\n!pip install sagemaker\n\n!pip install s3fs\n!pip install smdebug'

In [2]:
import sys
sys.path.append('../../../')

import os
import sys
import logging
import argparse
from pathlib import Path
from typing import Any, Dict, Optional

In [3]:
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

import sagemaker
from sagemaker import get_execution_role
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger


import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc

import pandas as pd

Local constants, regarding the data, MLFlow server, paths, etc..: use them

In [4]:
from deep.constants import *
from deep.utils import *

In [5]:
%load_ext autoreload
%autoreload 2

# Data

You can use the data you want. We advise the `pandas` format.

In [6]:
DATA_PATH = os.path.join(
    '..', '..', '..', "data", "frameworks_data", 'data_v0.6.2','generated_entries'
)


original_df = pd.read_csv(os.path.join(DATA_PATH, 'full_dataset.csv'))
augmented_data = pd.read_csv(os.path.join(DATA_PATH, 'generated_text.csv'))

/home/selim/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
test_data = pd.read_excel(os.path.join('..', '..', '..', '..', 'feedback_output.xlsx'))
test_data = test_data[['Entry']].rename(columns={'Entry':'excerpt'})
test_data = test_data[test_data.excerpt.apply(lambda x: 'NONE' != x.upper())]

In [8]:
augmented_data = pd.merge(
    right=original_df.drop(columns=['excerpt']),
    left=augmented_data[['entry_id', 'excerpt']],
    on='entry_id',
    how='right'
)

In [9]:
tot_df = pd.concat([original_df, augmented_data])

In [10]:
tot_df.shape

(489869, 21)

In [11]:
tot_df = tot_df[
    ['entry_id', 'excerpt', 'lead_id',
     'sectors',
     #'severity',
     #'demographic_groups',
     #'subpillars_1d', 
     #'specific_needs_groups',
     #'subpillars_2d', 
     #'affected_groups',
     
    ]
]

## Sagemaker Prep

### Session

Configure SageMaker

In [12]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
tracking_uri = MLFLOW_SERVER

### Bucket upload

You need to upload data to an S3 bucket. 




In [13]:
MLFLOW_SERVER

'http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/'

In [14]:
sample = True  # To make the computations faster, sample = True.

if sample:
    tot_df = tot_df.sample(n=5_000)
    
job_name = f"pytorch-{formatted_time()}-all-models"  # change it as you prefer
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name  # Do not change this

train_path = str(input_path / 'train.pickle')
val_path = str(input_path / 'val.pickle')


tot_df.to_pickle(train_path, protocol=4)  # protocol 4 is necessary, since SageMaker uses python 3.6
test_data.to_pickle(val_path, protocol=4)

### Estimator Definition

In [15]:
# GPU instances

instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

The hyperparameters are passed as command line arguments to the training script. 

You can add/change them as you like. It's important to keep the `tracking_uri` and the `experiment_name` which are used by MLFlow.

The class `PyTorch` is part of the `SageMaker` python API. The parameters are important and you should probably not change most of them. The ones you may want to change are:

- `instance_type`, specify the instance you want
- `source_dir`, specify your script directory. Try to use global variable as much as possible

In [16]:
from sagemaker.pytorch import PyTorch

proportions_negative_examples_test = {
    'sectors':0.16,
    'subpillars_2d':0.3,
    'subpillars_1d': 0.69,
    'demographic_groups': 0.74,
    'specific_needs_groups': 0.86,
    'affected_groups': 0.35
}
factor_prop_tot_train = 0
proportions_negative_examples_train = {
    key:value*factor_prop_tot_train for key, value in proportions_negative_examples_test.items()
}

hyperparameters={
    'tracking_uri': MLFLOW_SERVER,
    'experiment_name': "pl-trials",
    'max_len': 256,
    'epochs': 1,
    'model_name': 'microsoft/xtremedistil-l6-h256-uncased',
    'tokenizer_name': 'microsoft/xtremedistil-l6-h256-uncased',
    'dropout_rate': 0.3,
    'pred_threshold':0.4,
    'output_length': 256,
    'learning_rate': 5e-5,
    #'training_names':'sectors,subpillars_2d,subpillars_1d,specific_needs_groups,affected_groups,demographic_groups',
    'training_names':'sectors',
    #'training_names':'subpillars_1d,specific_needs_groups,demographic_groups',
    #'training_names':'sectors,subpillars_2d,subpillars_1d,severity,specific_needs_groups,affected_groups,demographic_groups',
    #'train_with_all_positive_examples':True,
    "model_mode":"train",
    "proportions_negative_examples_test": str(proportions_negative_examples_test),
    "proportions_negative_examples_train": str(proportions_negative_examples_train)
    #"numbers_augmentation":"with"
}

estimator = PyTorch(
    entry_point='train_mlflow.py',
    source_dir=str('../../../scripts/training/selim/multiclass-lightning'),
    output_path=str(DEV_BUCKET/'models/'),
    code_location=str(input_path),
    instance_type='ml.p2.xlarge',
    instance_count=1,
    role=role,
    framework_version="1.8",
    py_version="py36",
    hyperparameters = hyperparameters,
    job_name=job_name
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
)

In [17]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [18]:
# Fit the estimator

estimator.fit(fit_arguments, job_name=job_name)

2021-10-21 10:36:31 Starting - Starting the training job...
2021-10-21 10:37:00 Starting - Launching requested ML instancesProfilerReport-1634812588: InProgress
......
2021-10-21 10:38:00 Starting - Preparing the instances for training.........
2021-10-21 10:39:41 Downloading - Downloading input data...
2021-10-21 10:40:01 Training - Downloading the training image...........................
2021-10-21 10:45:02 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-10-21 10:44:54,579 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-10-21 10:44:54,602 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-10-21 10:44:54,613 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-10-21 10:44:55,344 sagemaker-training-toolkit INFO     Installing depende

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for nltk: filename=nltk-3.2.5-py3-none-any.whl size=1392143 sha256=28b6c06453b01131f0243f16fe8609feea1ba157a1b215abdf4a4f89ccd95349
  Stored in directory: /root/.cache/pip/wheels/f2/7f/71/cb36468789a03b5e2908281c8e1ce093e6860258b6b61677d8
  Created wheel for sagemaker: filename=sagemaker-2.49.1-py2.py3-none-any.whl size=591916 sha256=8f72225d028d1f2956f26b20d3397966c7dc05a6434ef69b61f92f80dbba8bd9
  Stored in directory: /root/.cache/pip/wheels/4c/af/ea/8ff5943a87155df5b184e54474fbf2b59b75e5c172854643c6
  Created wheel for aiobotocore: filename=aiobotocore-1.4.2-py3-none-any.whl size=49910 sha256=b26bf26911507d1541ddba1c522b6c752acdbf88e31715179dfabf2e0dad6049
  Stored in directory: /root/.cache/pip/wheels/28/61/17/b5ccee30498ab9e21ed6ed0f65d71f79cd252017bc95b6fc00
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158155 sha256=ffad02944f90b9f

  Created wheel for wrapt: filename=wrapt-1.12.1-cp36-cp36m-linux_x86_64.whl size=69744 sha256=120adbae407bf12f92b2c952e4b72e45f236b45c8de5b151c465d7a8414b3236
  Stored in directory: /root/.cache/pip/wheels/32/42/7f/23cae9ff6ef66798d00dc5d659088e57dbba01566f6c60db63
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-py3-none-any.whl size=3160 sha256=3c64b493c1efc6848314b4bcf6aa2b5bff6ec44901910b3e1755e2dd668933c0
  Stored in directory: /root/.cache/pip/wheels/6a/f5/9c/f8331a854f7a8739cf0e74c13854e4dd7b1af11b04fe1dde13
Successfully built nltk sagemaker aiobotocore alembic databricks-cli termcolor wrapt idna-ssl
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
 

#015Validation sanity check: 0it [00:00, ?it/s]#015Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s][2021-10-21 10:47:14.690 algo-1:86 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-10-21 10:47:14.734 algo-1:86 INFO profiler_config_parser.py:102] User has disabled profiler.
[2021-10-21 10:47:14.735 algo-1:86 INFO json_config.py:91] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
[2021-10-21 10:47:14.736 algo-1:86 INFO hook.py:201] tensorboard_dir has not been set for the hook. SMDebug will not be exporting tensorboard summaries.
[2021-10-21 10:47:14.737 algo-1:86 INFO hook.py:255] Saving to /opt/ml/output/tensors
[2021-10-21 10:47:14.737 algo-1:86 INFO state_store.py:77] The checkpoint config file /opt/ml/input/config/checkpointconfig.json does not exist.
[2021-10-21 10:47:14.937 algo-1:86 INFO hook.py:594] name:model.l0.embeddings.word_embeddings.weight count_params:7813632
[2021-10-21 10:47:14.938 algo-1:86 INFO hook.py:594] name

#015                                                              #015#015Training: 0it [00:00, ?it/s]#015Training:   0%|          | 0/80 [00:00<?, ?it/s]#015Epoch 0:   0%|          | 0/80 [00:00<?, ?it/s] #015Epoch 0:  38%|███▊      | 30/80 [00:06<00:10,  4.88it/s]#015Epoch 0:  38%|███▊      | 30/80 [00:06<00:10,  4.88it/s, loss=3.56, v_num=0, val_f1_epoch=0.071, val_loss_epoch=0.687, train_f1=0.535]#015Epoch 0:  38%|███▊      | 30/80 [00:19<00:32,  1.54it/s, loss=3.56, v_num=0, val_f1_epoch=0.071, val_loss_epoch=0.687, train_f1=0.535]#015Epoch 0:  75%|███████▌  | 60/80 [00:32<00:10,  1.87it/s, loss=3.56, v_num=0, val_f1_epoch=0.071, val_loss_epoch=0.687, train_f1=0.535]#015Epoch 0:  75%|███████▌  | 60/80 [00:32<00:10,  1.87it/s, loss=2.44, v_num=0, val_f1_epoch=0.071, val_loss_epoch=0.687, train_f1=0.620]#015Epoch 0: 100%|██████████| 80/80 [00:34<00:00,  2.31it/s, loss=2.44, v_num=0, val_f1_epoch=0.071, val_loss_epoch=0.687, train_f1=0.620]
#015Validating: 0it [00:00, ?it/s]#033[A
#0


2021-10-21 10:48:07 Uploading - Uploading generated training model
2021-10-21 10:48:44 Completed - Training job completed
ProfilerReport-1634812588: IssuesFound
Training seconds: 538
Billable seconds: 538


In [19]:
assert (1==2)


AssertionError: 

In [ ]:
import mlflow
import torch

In [ ]:
model = mlflow.pytorch.load_model(
    's3://deep-mlflow-artifact/16/206778d9e1584881b4f71f77b03ec27c/artifacts/pytorch_model_all',
    map_location=torch.device('cpu')   
)

In [ ]:
model.predict([], test_data['excerpt'])

In [ ]:
s = "{'ab':1}"

In [ ]:
from ast import literal_eval
literal_eval(s)

In [ ]:
import copy